In [ ]:
import requests
import time
import sys
import pandas as pd
import threading
import numpy as np
import time
import random
import sys


from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime, timedelta
from time import sleep
from tqdm.notebook import tqdm

##   РИА Новости

In [ ]:
from datetime import datetime, timedelta
def get_ria_links():
    base_url = "https://ria.ru/services/search/getmore/"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Referer": "https://ria.ru/",
        "Accept-Language": "ru-RU,ru;q=0.9"
    }
    links = []
    
    start_date = datetime(2025, 1, 1)
    end_date = datetime(2025, 5, 23)
    delta = timedelta(days=2)
    
    while start_date < end_date:
        next_date = min(start_date + delta, end_date)
        params = {
            "query": "новости",
            "date_from": start_date.strftime("%Y-%m-%d"),
            "date_to": next_date.strftime("%Y-%m-%d"),
            "sort": "date",
            "offset": 0
        }
        
        while True:
            response = requests.get(base_url, params=params, headers=headers)
            if response.status_code != 200:
                break
            soup = BeautifulSoup(response.text, "html.parser")
            new_links = [a["href"] for a in soup.find_all("a", class_="list-item__title")]
            if not new_links:
                break
            links.extend(new_links)
            params["offset"] += len(new_links)
            time.sleep(4)  
        
        start_date = next_date + timedelta(days=1)
    
    return links



ria_links = get_ria_links()


print("РИА Новости:", len(ria_links), "ссылок")

In [ ]:
def GetArticle_ria(url0):
    """
    Возвращает кортеж с URL, датой публикации, заголовком, тегами, рубрикой и текстом статьи.
    """
    page0 = requests.get(url0)
    if page0.status_code != 200:
        return url0, None, None, None, None, None 

    soup0 = BeautifulSoup(page0.text, 'html.parser')

    title = soup0.find('meta', attrs={'name': 'analytics:title'})
    tag = soup0.find('meta', attrs={'name': 'analytics:tags'})
    rubric = soup0.find('meta', attrs={'name': 'analytics:rubric'})

    title = title['content'] if title else None
    tag = tag['content'] if tag else None
    rubric = rubric['content'] if rubric else None

    article_texts = soup0.find_all('div', class_='article__text')
    final_text = " ".join([text.get_text(strip=True) for text in article_texts]) if article_texts else None

    date_published = None
    time_tag = soup0.find(attrs={"data-published-time": True})

    if time_tag:
        raw_date = time_tag["data-published-time"]  
        try:
            parsed_date = datetime.strptime(raw_date, "%Y%m%dT%H%M")  
            date_published = parsed_date.strftime("%Y-%m-%d %H:%M:%S") 
        except ValueError:
            date_published = None 
    
    return url0, date_published, title, tag, rubric, final_text

In [ ]:
news_ria = [] 

for lin in ria_links:
    res = GetArticle_ria(lin)
    news_ria.append(res)
        
    sleep(4)
    
df = pd.DataFrame(news_ria)
df.head()

In [ ]:
df.info()

In [ ]:
df.to_csv("ria1.csv", index=False, encoding="utf-8")
print("Файл ria.csv сохранен.")

##   Лента.ру

In [ ]:
def get_lenta_links():
    base_url = "https://lenta.ru/parts/news/"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}
    links = []
    page = 1
    total_pages = 10
    while page <= total_pages:
        response = requests.get(base_url + f"{page}/", headers=headers)
        if response.status_code != 200:
            break
        soup = BeautifulSoup(response.text, "html.parser")
        new_links = ["https://lenta.ru" + a["href"] for a in soup.find_all("a", href=True) if a["href"].startswith("/news/")]
        if not new_links:
            break
        links.extend(new_links)
        page += 1
        time.sleep(1)
    print()
    return links
lenta_links = get_lenta_links()
print("Лента.ру:", len(lenta_links), "ссылок")

In [ ]:
def GetArticle_lenta(url0):
    """
    Возвращает кортеж с URL, датой публикации, заголовком, тегами, рубрикой и текстом статьи.
    """
    page0 = requests.get(url0)
    if page0.status_code != 200:
        return url0, None, None, None, None, None 

    soup0 = BeautifulSoup(page0.text, 'html.parser')

    rubric_tag = soup0.find('div', class_='rubric-header__title')
    rubric = rubric_tag.get_text(strip=True) if rubric_tag else None
    
    title_i = soup0.find('span', class_='topic-body__title')
    title = title_i.get_text(strip=True) if title_i else None
    
    tag = None
    
    authors_i = soup0.find('span', class_='topic-authors__name')
    authors = authors_i.get_text(strip=True) if authors_i else None

    article_texts = soup0.find_all('p', class_='topic-body__content-text')
    final_text = " ".join([text.get_text(strip=True) for text in article_texts]) if article_texts else None

    date_published = None
    time_tag = soup0.find("a", class_="topic-header__item topic-header__time")

    if time_tag:
        raw_date = time_tag.get_text(strip=True)  

       
        month_translate = {
            "января": "January", "февраля": "February", "марта": "March",
            "апреля": "April", "мая": "May", "июня": "June",
            "июля": "July", "августа": "August", "сентября": "September",
            "октября": "October", "ноября": "November", "декабря": "December"
        }

        
        parts = raw_date.split()
        if len(parts) == 4:  
            parts[2] = month_translate.get(parts[2], parts[2]) 
            clean_date = " ".join(parts)  

            try:
                parsed_date = datetime.strptime(clean_date, "%H:%M, %d %B %Y")
                date_published = parsed_date.strftime("%Y-%m-%d %H:%M:%S")
            except ValueError as e:
                print(f"Ошибка при разборе даты: {e}")
                date_published = None
    
    return url0, date_published, title, tag, rubric, final_text, authors

In [ ]:
news_lenta = [] 

for lin in lenta_links:
    res = GetArticle_lenta(lin)
    news_lenta.append(res)
        
    sleep(2)
    
df = pd.DataFrame(news_lenta)
df.head()

In [ ]:
df.to_csv("lenta1.csv", index=False, encoding="utf-8")
print("Файл lenta.csv сохранен.")

##   РБК

In [ ]:
def get_rbk_links():
    base_url = "https://www.rbc.ru/short_news"
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(base_url)
    time.sleep(2)
    
    links = set()
    scroll_pause_time = 8
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:

        soup = BeautifulSoup(driver.page_source, "html.parser")
        for a in soup.find_all("a", href=True):
            href = a["href"]
            if "https://www.rbc.ru/" in href:
                links.add(href)
        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)
        
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    driver.quit()
    return list(links)

rbk_links = get_rbk_links()

In [ ]:
def GetArticle_rbk(url0):
    """
    Возвращает кортеж с url0, date, author, description, title, final_text, rubrics.
    """
    try:
        page0 = requests.get(url0)
        page0.raise_for_status()
    except requests.RequestException as e:
        print(f"Ошибка при запросе страницы: {e}")
        return None

    soup0 = BeautifulSoup(page0.text, 'html.parser')

    try:
        tag = soup0.find('meta', attrs={'name': 'news_keywords'}).get('content')
    except AttributeError:
        tag = None

    try:
        title = soup0.find('meta', {'property': 'og:title'}).get('content')
    except AttributeError:
        title = None

    try:
        rubric = soup0.find('meta', {'property': 'og:site_name'}).get('content')
    except AttributeError:
        rubric = None

    try:
        text = soup0.find_all('p')
        cleaned_text = [element.get_text().strip() for element in text]
        final_text = ' '.join(cleaned_text)
    except AttributeError:
        final_text = None

    try:
        time_tag = soup0.find("time", class_="article__header__date")
        if time_tag and "datetime" in time_tag.attrs:
            iso_date = time_tag["datetime"]
            parsed_date = datetime.fromisoformat(iso_date)
            date = parsed_date.strftime("%Y-%m-%d %H:%M:%S")
        else:
            date = None
    except AttributeError:
        date = None

    return url0, date, tag, title, rubric, final_text

In [ ]:
news_rbk = []

for lin in rbk_links:
    res = GetArticle_rbk(lin)
    news_rbk.append(res)
        
    sleep(2)
    
df = pd.DataFrame(news_rbk)
df.head()

In [ ]:
df.to_csv("rbk1.csv", index=False, encoding="utf-8")
print("Файл rbk.csv сохранен.")